In [1]:
import numpy as np
from scipy.integrate import odeint, solve_ivp
import pandas as pd
import plotly.graph_objs as go
from plotly.subplots import make_subplots

In [2]:
    qsmax=      1.25270343 
    mumaxE=     0.17 
    base_coef=  0.53168754 
    cSCrab=     0.1 
    Ks=         0.1 
    Ke=         0.1 
    Ki=         0.1 
    YxsOx=      0.49 
    YxsRed=     0.05 
    Yxe=        0.72 
    YexRed=     9.58516 
    YesRed=     0.4792 
    Yco2xRed=   9.244 
    Yco2xOx=    1.233 
    Yco2xEt=    0.8957472 

In [3]:
# neu wrapper rumprobieren

csf = 200
feed_rate = 0.0069

dV_gas_dt = 30          # l/h
R = 0.08314           #bar*l/mol*K
T = 32 + 273.15         #Kelvin
p = 1.0177848837209327  #bar
M_CO2 = 44.01           #g/mol



def model_rhs(t,y):   
    
    global inner_func       #for accessing CO"% later on
    def inner_func(t,y):
        Fin = feed_rate * (t > 0.3833333333604969)

        mS, mX, mE, V  = y[0:4]
        cS, cX, cE = np.array([mS, mX, mE]) / V
        
        qs = qsmax * cS / (cS + Ks)
        cSCrab = 0.1

        if cS > cSCrab:
            qsOx = qsmax  * cSCrab / (cSCrab + Ks)
            qsRed = qs - qsOx 
            muE = 0.0
            
        else:
            qsOx = qs  
            qsRed = 0
            muE = mumaxE* cE/(cE + Ke) * Ki / (cS+ Ki)

        muOx = qsOx * YxsOx
        muRed = qsRed * YxsRed
        qE = 1/Yxe * muE

        dmS_dt = cX *V*(- qsOx- qsRed)+ csf *Fin
        dmX_dt = (muOx+muRed+muE) * cX * V
        
            
        dmE_dt = (qsRed * YesRed - qE)*cX*V

        dnAbgas_dt = p * dV_gas_dt / (R*T)

        dmCO2_dt = (muRed* Yco2xRed + muOx*Yco2xOx + muE * Yco2xEt) *cX*V
        dnCO2_dt = dmCO2_dt/M_CO2
        
        CO2_percent = 100 * dnCO2_dt / dnAbgas_dt 
        dV_dt  = + Fin
        
        return dmS_dt, dmX_dt, dmE_dt,  dV_dt, CO2_percent

    dmS_dt, dmX_dt, dmE_dt,  dV_dt, CO2_percent = inner_func(t,y)

    return dmS_dt, dmX_dt, dmE_dt,  dV_dt
        
        

tModell = np.linspace(0, 9, 1001)
y0 = [5.0, 0.672185, 0, 0.5]

result = solve_ivp(model_rhs, [np.min(tModell), np.max(tModell)], y0, t_eval=tModell, method= "Radau", first_step = 0.0000001).y.T

mS, mX, mE, V = [result[:,i] for i in range(4)]
cS, cX, cE = np.array([mS, mX, mE]) / V


CO2_percent = np.array([inner_func(tModell[i], result[i,:]) for i in range(len(tModell))])[:,4]



df = pd.DataFrame(
          {'t': tModell,
          'cS': cS, 'cX': cX,
          'V': V,'CO2' : CO2_percent, 'cE' : cE }
          ).set_index('t')

In [4]:
line_markers = "lines+markers"
line = "lines"



    
fig = make_subplots(specs=[[{"secondary_y": True}]])    
for column in df:
    if column in ["cS", "cX", "cE", "CO2", ]:

        secondary_y_flag = column  in ["mE", "CO2%"]                                       #or "CO2"
        
        fig.add_trace( 
    go.Scatter(x= df.index, y= df[column], name= column, mode = line), 
    secondary_y= secondary_y_flag,
    )
    
        
fig.show()